In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy
import torch
from tqdm import tqdm

import sys
sys.path.append('../../')
from mcabc.model.PoissonModel import PoissonModel
from mcabc.model.NegativeBinomialModel import NegativeBinomialModel
from mcabc.mdn.Trainer import Trainer
from mcabc.mdn.MixtureDensityNetwork import ClassificationMDN, UnivariateMogMDN, MultivariateMogMDN

from mcabc.utils.stats import calculate_nb_evidence, poisson_evidence, calculate_nb_evidence, calculate_pprob_from_evidences
from mcabc.utils.processing import generate_poisson_nb_data_set, calculate_stats_toy_examples, normalize


#from delfi.distribution.mixture import MoG
%matplotlib inline

## Set up Poisson and NB model

In [ ]:
# number of samples per data point
sample_size = 10
# number of training data points
ntrain = 100000
# number of testing data points
ntest = 100

# prior hyper parameters : k2 small --> difficult, k2 large --> easy
# NB model Gamma shape
k2 = 20
theta2 = 1.0
# NB model Gamma scale
k3 = 2.
theta3 = 2.
# Poisson model Gamma prior scale
theta1 = 2.0
# set Gamma prior shape such that the expected sample means of both model are equal:
k1 = (k2 * theta2 * k3 * theta3) / theta1

# define the models
model_poisson = PoissonModel(sample_size=sample_size)
model_nb = NegativeBinomialModel(sample_size=sample_size)

## Generate parameters from the priors

In [ ]:
# define the prior with the above hyper parameters
prior_lam = scipy.stats.gamma(a=k1, scale=theta1) # gamma prior for Poisson model
prior_k = scipy.stats.gamma(a=k2, scale=theta2) # gamma prior on Gamma distribution shape for the NB model
prior_theta = scipy.stats.gamma(a=k3, scale=theta3) # gamma prior on Gamma distribution scale for the NB model

# draw params from priors
n = ntrain + ntest
params_poi = prior_lam.rvs(size=int(n / 2))
params_nb = np.vstack((prior_k.rvs(size=int(n / 2)),
                       prior_theta.rvs(size=int(n / 2)))).T

## Generate data from models and calculate summary stats, prepare test data

In [ ]:
# generate all data
data_poi = model_poisson.gen(params_poi)
data_nb = model_nb.gen(params_nb)

In [ ]:
# shuffle and set up model index target vector
x_all = np.vstack((data_poi, data_nb))

# define model indices
m_all = np.hstack((np.zeros(data_poi.shape[0]), np.ones(data_nb.shape[0]))).squeeze().astype(int)

# get shuffled indices
shuffle_indices = np.arange(n)
np.random.shuffle(shuffle_indices)

# shuffle the data
x_all = x_all[shuffle_indices, ]
m_all = m_all[shuffle_indices].tolist()

# separate into training and testing data
x, xtest = x_all[:ntrain, :], x_all[ntrain:, :]
m, mtest = m_all[:ntrain], m_all[ntrain:]

# calculate summary stats
sx = calculate_stats_toy_examples(x)
sx_test = calculate_stats_toy_examples(xtest)

# use training norm to normalize test data
sx_zt, training_norm = normalize(sx)
sx_test_zt, training_norm = normalize(sx_test, training_norm)

## Set up the MDN and train it

In [ ]:
model = ClassificationMDN(n_input=2, n_hidden_units=10, n_hidden_layers=1)
trainer = Trainer(model,
                  optimizer=torch.optim.Adam(model.parameters(), lr=0.01), verbose=True, classification=True)

n_epochs = 100
n_minibatch = int(ntrain / 100)

# train with training data
loss_trace = trainer.train(sx_zt, m, n_epochs=n_epochs, n_minibatch=n_minibatch)
plt.figure(figsize=(18, 5))
plt.plot(loss_trace)
plt.ylabel('loss')
plt.xlabel('iterations');

## Calculate the exact posterior probabilities for test data

In [ ]:
ppoi_exact = []
# progress bar
with tqdm(total=len(mtest), desc='Calculate evidences for test samples', ncols=110) as pbar:
    # for every test sample
    for xi in xtest:
        # calcuate the evidences
        nb_logevi = calculate_nb_evidence(xi, k2, theta2, k3, theta3, log=True)
        poi_logevi = poisson_evidence(xi, k=k1, theta=theta1, log=True)
        # calculate the posterior prob from the model evidences, given the prior is uniform.
        ppoi_exact.append(calculate_pprob_from_evidences(np.exp(poi_logevi), np.exp(nb_logevi)))
        pbar.update()

## Visualize the MDN input output function

In [ ]:
# set up a grid of means and variances as input to the MDN
ms, vs = np.meshgrid(np.linspace(0, 300, 100), np.linspace(0, 300, 100))
# stack values to evaluate as vector in the model
sx_vis = np.vstack((ms.flatten(), vs.flatten())).T
# normalize
sx_vis, training_norm = normalize(sx_vis, training_norm)

# predict probs
ppoi_vec = model.predict(sx_vis)
# take poisson posterior prob and reshape to grid dimensions
ppoi_vismat = ppoi_vec[:, 0].reshape(ms.shape[0], vs.shape[0])

In [ ]:
plt.figure(figsize=(18, 5))
# define color norm
cnorm = mpl.colors.Normalize(vmin=ppoi_vismat.min(), vmax=ppoi_vismat.max())
cmap = plt.cm.viridis

# plot test data points with true posterior probability as color code
plt.scatter(x=sx_test[:, 0], y=sx_test[:, 1], c=np.array(ppoi_exact), cmap=cmap,
            norm=cnorm, edgecolors='r', linewidths=0.3)
# plot predicted posterior probs for the entire grid with same color code
plt.imshow(ppoi_vismat, origin='lower', aspect='auto',
           norm=cnorm, cmap=cmap, extent=[ms.min(), ms.max(), vs.min(), vs.max()])
plt.xlabel('Sample mean', fontsize=15)
plt.ylabel('Sample variance', fontsize=15)
plt.colorbar(label='P(Poisson | x)', pad=0.01)
plt.legend(['Exact posterior probabilities'], fontsize=15)
plt.tight_layout();

## Observe some Poisson data and predict the posterior probability

In [ ]:
# generate a single sample of Poisson data, with lambda at the mean of the model prior
true_lambda = scipy.stats.gamma.rvs(a=k1, scale=theta1)
x_obs = scipy.stats.poisson.rvs(mu=true_lambda, size=sample_size)
# calculate stats
sx_obs = calculate_stats_toy_examples(x_obs)
# normalize using training data normalization
sx_obs_zt, training_norm = normalize(sx_obs, training_norm)

# predict
p_vec = model.predict(sx_obs_zt).squeeze()
# calculate exact evidence
nb_evidence = calculate_nb_evidence(x_obs, k2, theta2, k3, theta3, log=True)
poi_evidence = poisson_evidence(x_obs, k=k1, theta=theta1, log=True)
ppoi_ana = calculate_pprob_from_evidences(np.exp(poi_evidence), np.exp(nb_evidence))

print(r'predicted P(poisson | data) = {:.2f}'.format(p_vec[0]))
print(r'exact P(poisson | data) = {:.2f}'.format(ppoi_ana))

## Given the high posterior probability for the Poisson model we predict the posterior over its parameters

In [ ]:
# define a network to approximate the posterior with a MoG
model_params_mdn = UnivariateMogMDN(ndim_input=2, n_hidden=10, n_components=2)
optimizer = torch.optim.Adam(model_params_mdn.parameters(), lr=0.01)
trainer = Trainer(model_params_mdn, optimizer, verbose=True)

# calculate stats for poisson model
sx_poi = calculate_stats_toy_examples(data_poi)
# normalize data
sx_poi_zt, data_norm = normalize(sx_poi)
# normalize prior params
params_poi_zt, prior_norm = normalize(params_poi)

loss_trace = trainer.train(sx_poi_zt, params_poi_zt, n_epochs=100, n_minibatch=int(n_minibatch))
plt.figure(figsize=(18, 5))
plt.plot(loss_trace)
plt.ylabel('loss')
plt.xlabel('iterations');

In [ ]:
# normalize the observed data
sx_obs_zt, data_norm = normalize(sx_obs, data_norm)

## Predict posterior for observed data

In [ ]:
# predicte the posterior for the observed data, this is in normalized parameter space
predicted_posterior = model_params_mdn.predict(sx_obs_zt)
# transform back to actual parameter space
predicted_posterior = predicted_posterior.ztrans_inv(mean=prior_norm[0], std=prior_norm[1])

## Calculate analytical posterior for observed data

In [ ]:
# get analytical gamma posterior
k_post = k1 + np.sum(x_obs)

# use the posterior given the summary stats, not the data vector
scale_post = 1. / (sample_size + theta1**-1)
exact_posterior = scipy.stats.gamma(a=k_post, scale=scale_post)

# create a range of parameters for plotting the two posteriors
thetas = np.linspace(true_lambda - 10, true_lambda + 10, 1000)

In [ ]:
plt.figure(figsize=(18, 5))
plt.title(r'Gamma posterior fit for Poisson $\lambda$')

# plt.plot(thetas_poisson, gamma_prior.pdf(thetas_poisson), label='prior')
plt.plot(thetas, exact_posterior.pdf(thetas), label='analytical posterior')
plt.plot(thetas, predicted_posterior.eval_numpy(thetas), label='predicted posterior')
plt.axvline(x=k1 * theta1, color="C2", label='true lambda')
plt.legend(fontsize=15)
plt.xlabel(r'$\lambda$', fontsize=15)
plt.ylabel('Posterior density', fontsize=15);

## We can also observe NB data and learn the posterior over the parameters of the NB model:
The NB posterior is 2D, one dimension for $k$ and once for $\theta$.

In [ ]:
# define a network to approximate the posterior with a MoG
model_params_mdn = MultivariateMogMDN(ndim_input=2, ndim_output=2, n_hidden_units=10,
                                      n_hidden_layers=2, n_components=2)
optimizer = torch.optim.Adam(model_params_mdn.parameters(), lr=0.01)
trainer = Trainer(model_params_mdn, optimizer, verbose=True)

In [ ]:
# calculate stats for poisson model
sx_nb = calculate_stats_toy_examples(data_nb)
# normalize data
sx_nb_zt, data_norm = normalize(sx_nb)
# normalize prior params
params_nb_zt, prior_norm = normalize(params_nb)

loss_trace = trainer.train(sx_nb_zt, params_nb_zt, n_epochs=200, n_minibatch=200)
plt.figure(figsize=(18, 5))
plt.plot(loss_trace)
plt.ylabel('loss')
plt.xlabel('iterations');

In [ ]:
# observe new NB data, normalize with norms from training data
thetao = [[prior_k.rvs(), prior_theta.rvs()]]
xo = model_nb.gen(thetao)
sxo = calculate_stats_toy_examples(xo)
sxo_zt, _ = normalize(sxo, data_norm)
thetao_zt, _ = normalize(thetao, prior_norm)

In [ ]:
# predict posterior and transform to absolut range
predicted_posterior_nb = model_params_mdn.predict(sxo_zt).ztrans_inv(mean=prior_norm[0], std=prior_norm[1])

## Calculate exact NB posterior
Because the NB distribution as defined here is not in the exponential family, there is no closed form posterior. We approximate the exact solution numerically and visualize the resulting posterior as a 2D histogram.